# Predict

This is the notebook for loading and training models.
Furthermore it provides simple documentation for different approaches used for training a model.

Run the command below to see command-completion on pressing `TAB`.

### Imports

In [47]:
# Imports
import os
import warnings
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras import layers, models
from matplotlib import pyplot
import numpy as np
from sklearn.metrics import confusion_matrix
import sklearn
import wandb
from data_repository import DataRepository

# Ignore future warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [48]:
# Root CSV files directory
#dirname = "./data/absolute/live/"
dirname = "./data/absolute/2D/"
#dirname = "./data/absolute/modified/" 

repo = DataRepository(dirname, verbose=False)

x, y = repo.getDataAndLabels()

# Load data and print summary, if desired
#x_train, x_val, x_test, y_train, y_val, y_test, labels = repo.getForTraining()

['Computer', 'Deutschland', 'du', 'Haben', 'Hallo', 'ich', 'Mainz', 'Software', 'unser', 'Welt', 'zeigen']


### Load Data

In [55]:

#for i in range(0, len(x)):
#   print(i, " " , x[i][2][2])

print(x[0][3][2])



x.shape

#for i in range(0, len(x_train[7])):
#    print(i, " " , x_train[7][i])


0.351603


(684, 100, 86)

In [54]:
with open('tokens_json.txt', 'r') as outfile:
    json_ex = outfile.read()
    

tokenizer = tf.keras.preprocessing.text.tokenizer_from_json(json_ex)
print(tokenizer.word_index)
token_labels = {y:x for x,y in tokenizer.word_index.items()}

{'computer': 1, 'deutschland': 2, 'du': 3, 'haben': 4, 'hallo': 5, 'ich': 6, 'mainz': 7, 'software': 8, 'unser': 9, 'welt': 10, 'zeigen': 11}


In [39]:
physical_devices = tf.config.list_physical_devices('GPU') 
print("Num GPUs:", len(physical_devices)) 

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.3
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

Num GPUs: 1


/home/michi/anaconda3/envs/tensor-gpu/lib/python3.8/site-packages/tensorflow/python/client/session.py:1751: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [40]:
model = tf.keras.models.load_model('model-best_2d.h5')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100, 64)           38656     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 256)          328704    
_________________________________________________________________
lstm_2 (LSTM)                (None, 100, 256)          525312    
_________________________________________________________________
lstm_3 (LSTM)                (None, 100, 256)          525312    
_________________________________________________________________
lstm_4 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dense (Dense)                (None, 12)                3084      
Total params: 1,946,380
Trainable params: 1,946,380
Non-trainable params: 0
______________________________________________

### Evaluate

In [41]:
eval = model.evaluate(x, y, verbose=2)

1/1 - 0s - loss: 4.5792 - accuracy: 0.2500 - precision: 0.2500 - recall: 0.2500


In [44]:
predict = model.predict(x)
#print(predict)

y_pred = np.array([np.argmax(pred) for pred in predict])

y_pred_integer = np.argmax(predict, axis=1)
y_integer = np.argmax(y, axis=1)

print(y_pred_integer)

y_pred_name = ([token_labels[p] for p in y_pred_integer])
y_name = ([token_labels[p] for p in y_integer])


[ 5  5  8  5  5  5 10 10  9 10  2  7]


In [45]:
y_pred_name

['hallo',
 'hallo',
 'software',
 'hallo',
 'hallo',
 'hallo',
 'welt',
 'welt',
 'unser',
 'welt',
 'deutschland',
 'mainz']

In [46]:
y_name 

['hallo',
 'hallo',
 'hallo',
 'hallo',
 'zeigen',
 'zeigen',
 'zeigen',
 'zeigen',
 'zeigen',
 'zeigen',
 'zeigen',
 'zeigen']

In [ ]:
wandb.init(project="Test", name="confusion")

In [ ]:
# Confusion Matrix
#sklearn.metrics.plot_confusion_matrix(y_test_name, y_pred_name)
wandb.sklearn.plot_confusion_matrix(y_train_name, y_pred_name)

# TFLITE

In [ ]:
x_test=np.array(x_test,dtype=np.float32)


print(type(x_train[0][0][0]))
print(type(x_test[0][0][0]))


In [ ]:
import numpy as np
import tensorflow as tf
from time import time

# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="sign_lang_recognition_2D.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print(input_details)


input_shape = input_details[0]['shape']
time1= time()
input_data = [1]
predictions = []

input_details[0]["index"]

for x_pred in x_test:
    
    input_data[0] = x_pred
    interpreter.set_tensor(input_details[0]['index'], input_data)

    interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
    output_data = interpreter.get_tensor(output_details[0]['index'])
    predictions.append(np.argmax(output_data))
    
predictions = [token_labels[p] for p in predictions]

In [ ]:
for i in range(0, len(predictions)):
    print('keras: ',y_pred_name[i],  predictions[i])

In [ ]:
print("Predictions:", predictions)
time2= time()
dauer = time2 - time1
print("Anzahl Samples: ", len(predictions))
print("Predictiondauer: ", dauer)
print("Samples pro Sekunde:", (len(predictions)/dauer))

In [ ]:
print(input_data)